In [12]:
""" In this Notebook I shall investigate using Osborne's PIGEBaQ method on 
the test data obtained from my permutation algorithm. We shall see if the results agree, 
and learn a bit about the challenges of using this approach."""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

df = pd.read_csv("/Users/jacob/Documents/4YP data/ONET_18-10-21/test.csv") #This is all the data I will need
df.drop(columns=['O*NET-SOC Code', 'Title'], inplace=True)
df.head()

,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,Learning Strategies,Monitoring,...,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources,Auto label value
0,4.75,4.88,4.38,4.88,3.62,1.12,4.75,4.75,3.88,5.50,...,0.0,1.00,5.75,5.38,5.12,4.75,5.50,4.75,5.38,0.0
1,4.25,4.00,4.25,4.12,3.12,1.88,4.12,3.88,3.75,4.00,...,0.0,1.50,4.00,4.00,4.00,3.88,3.12,2.62,4.00,NaN
2,4.00,4.00,3.88,4.00,2.50,1.12,4.00,3.62,3.25,4.00,...,0.0,2.12,3.75,3.00,3.12,3.75,3.38,3.25,3.88,NaN
3,4.00,4.12,3.88,4.12,3.25,0.62,4.12,4.12,3.25,4.25,...,0.0,1.00,4.00,3.12,3.75,3.88,3.62,2.62,3.88,NaN
4,4.25,4.12,3.88,4.12,3.12,1.50,4.25,4.12,3.50,4.25,...,0.0,1.38,4.00,3.75,3.75,3.75,3.75,2.75,3.88,NaN


In [14]:
#Lets now drop the unseen observations to leave just the training data:

df1 = df.dropna()
df1.reset_index(drop=True,inplace=True)
df1.head()

,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,Learning Strategies,Monitoring,...,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources,Auto label value
0,4.75,4.88,4.38,4.88,3.62,1.12,4.75,4.75,3.88,5.50,...,0.00,1.00,5.75,5.38,5.12,4.75,5.50,4.75,5.38,0.0
1,4.00,3.96,3.67,3.96,3.17,0.46,4.08,3.66,3.38,4.13,...,0.08,1.96,3.63,3.46,3.67,3.88,3.33,3.17,3.79,0.0
2,4.12,4.00,4.00,4.00,2.88,0.75,4.00,3.88,3.88,4.12,...,0.00,1.75,3.88,3.12,3.38,3.88,3.25,3.12,4.00,0.0
3,4.12,4.25,4.00,4.12,3.12,2.00,4.12,3.88,3.75,4.00,...,0.00,1.62,4.12,3.75,3.88,4.00,3.12,2.38,4.38,0.0
4,4.06,3.94,3.75,3.88,3.06,0.44,4.00,3.31,2.56,3.12,...,0.00,1.81,3.62,2.44,2.44,2.94,2.19,1.56,2.25,1.0


In [40]:
#We first need to model the input variables of our data. We shall assume they are gaussian (non-mixture)

mu = list(df.mean(axis='index'))
del mu[-1]
m = len(mu)
print(m)

n = len(df1.index)
print(n)

np_mu = np.array(mu)
print(np_mu)
#We have our vector of means (mx1)

35
63
[3.64368843 3.54613975 3.29475372 3.45728522 2.56115693 1.50342497
 3.57       3.19258877 2.84367698 3.37768614 3.10325315 3.14676976
 2.76437572 2.564937   2.91871707 2.85216495 3.15989691 1.73046964
 1.00459336 1.07781214 0.35969072 0.81539519 2.28172967 1.7741008
 1.05408935 1.56753723 1.02463918 2.17970218 3.20827033 2.63945017
 2.62533792 3.0472394  1.34610538 1.48531501 2.58720504]


In [16]:
#Now lets get our covariance matrix:

df2 = df1.drop(columns=['Auto label value']) #This dataframe removes the output of our GP so the covariance isn't effected

df_cov = df2.cov()
df_cov.head()

np_cov = df_cov.to_numpy()
print(np_cov)
#We have our covariance matrix (mxm)

[[0.76444782 0.58130456 0.69916557 ... 0.45318431 0.3671946  0.33895143]
 [0.58130456 0.56572002 0.57766101 ... 0.3614607  0.27286907 0.31405394]
 [0.69916557 0.57766101 0.70013492 ... 0.38176979 0.30487363 0.32323369]
 ...
 [0.45318431 0.3614607  0.38176979 ... 1.31053507 1.10093914 0.76044068]
 [0.3671946  0.27286907 0.30487363 ... 1.10093914 1.00709621 0.6898353 ]
 [0.33895143 0.31405394 0.32323369 ... 0.76044068 0.6898353  0.71434444]]


In [17]:
print(np.shape(np_mu))
print(np.shape(np_cov))

(35,)
(35, 35)


In [18]:
#Now lets focus on the matrix of lengthscales and the output variance of the kernel:

output_var = 125
lengthscale = 8  #both values estimated while get_params method continues to not work
np_lengthscale = lengthscale*np.identity(m)



In [19]:
#We now have all the ingredients we need to make z_n (for each observation):
#Lets begin by just focusing on the first observation, and then creating a function for this
import math

x = np.array(df2.iloc[[0]])

element1 = (output_var*math.sqrt(np.linalg.det(np_lengthscale)))/(math.sqrt(np.linalg.det(np_lengthscale+np_cov)))
#print(element1)
element2 = np.linalg.inv(np_lengthscale+np_cov)
#print(element2)
element3 = np.transpose(np_mu - x)
#print(element3)
#temp = np.matmul(element2,-1*element3)
element4 = -0.5*np.matmul(np.transpose(-1*element3),np.matmul(element2,-1*element3))
#print(element4)

z = element1*math.exp(element4.item())*np.matmul(element2,element3)
print(z)

[[-0.00860548]
 [-0.06894903]
 [-0.01170304]
 [-0.0725187 ]
 [-0.02920896]
 [ 0.22937517]
 [-0.03208581]
 [-0.07919282]
 [-0.01842026]
 [-0.22136611]
 [-0.0328072 ]
 [-0.20914795]
 [-0.1989016 ]
 [-0.18529898]
 [-0.02611171]
 [ 0.02813506]
 [-0.15039913]
 [-0.20639342]
 [ 0.1156125 ]
 [ 0.03784764]
 [ 0.0399778 ]
 [ 0.12448286]
 [ 0.09360752]
 [-0.02798726]
 [ 0.09913313]
 [ 0.20658357]
 [ 0.11353483]
 [ 0.17739136]
 [-0.26150596]
 [-0.26995348]
 [-0.20238852]
 [-0.16396686]
 [-0.48638356]
 [-0.36933394]
 [-0.31257482]]


In [20]:
#Lets now create a function that does this for each observation:

def compute_zn(index):
    x = np.array(df2.iloc[[index]])

    element1 = (output_var*math.sqrt(np.linalg.det(np_lengthscale)))/(math.sqrt(np.linalg.det(np_lengthscale+np_cov)))
    element2 = np.linalg.inv(np_lengthscale+np_cov)
    element3 = np.transpose(np_mu - x)
    element4 = -0.5*np.matmul(np.transpose(-1*element3),np.matmul(element2,-1*element3))

    zn = element1*math.exp(element4.item())*np.matmul(element2,element3)
    return zn


In [41]:
Z = [0]*n
for i in range(0,n):
    Z[i] = compute_zn(i)

Z = np.array(Z)
Z = Z.reshape(m,n)
print(Z)

[[-0.00860548 -0.06894903 -0.01170304 ...  0.76190741  1.61546301
   0.19852163]
 [ 0.05586835 -0.70885447 -1.11877989 ...  0.14882637  1.74613009
   0.50923952]
 [ 0.394528   -0.20829026 -0.29004295 ...  0.91202445 -0.87630489
  -1.100605  ]
 ...
 [ 0.77157932  0.44761931 -0.40216863 ...  2.04667063  0.72953125
   1.09449997]
 [ 1.97317995  0.32414552  1.364847   ... -0.511011    1.39508738
   0.54003225]
 [ 1.59337505  0.58907687  1.02868886 ... -0.28181533  0.23086294
   1.1566362 ]]


In [42]:
#We now need to find K, which is a matrix of the kernel outputs of the training data.
X = df2.to_numpy()
X_norm = np.sum(X ** 2, axis = -1)
K = output_var * np.exp(-(1/lengthscale) * (X_norm[:,None] + X_norm[None,:] - 2 * np.dot(X, X.T)))
print(K)
np.shape(K)

[[1.25000000e+02 8.79337705e-01 2.96228619e-01 ... 8.03007993e-08
  4.34522338e-10 3.73584772e-11]
 [8.79337705e-01 1.25000000e+02 4.59005104e+01 ... 5.24635696e-02
  9.47215570e-03 1.49783243e-03]
 [2.96228619e-01 4.59005104e+01 1.25000000e+02 ... 1.41156134e-02
  7.43236429e-03 4.24277107e-04]
 ...
 [8.03007993e-08 5.24635696e-02 1.41156134e-02 ... 1.25000000e+02
  7.05817366e-01 1.19479981e+01]
 [4.34522338e-10 9.47215570e-03 7.43236429e-03 ... 7.05817366e-01
  1.25000000e+02 1.66623426e+01]
 [3.73584772e-11 1.49783243e-03 4.24277107e-04 ... 1.19479981e+01
  1.66623426e+01 1.25000000e+02]]


(63, 63)

In [43]:
#Final ingredient: we need a row vector of the outputs:

f = df1["Auto label value"].to_numpy()
f = f.reshape(63)
np.shape(f)

(63,)

In [47]:
#Moment of truth...

E = np.matmul(Z,np.matmul(np.linalg.inv(K),f))
print(E)

[ 0.02866842  0.0823293  -0.16126411 -0.00660473  0.02496839  0.14476136
  0.0619657  -0.17035649  0.04403746 -0.02174297  0.13749662 -0.02743521
 -0.09849151  0.03519559  0.00281035 -0.05874347 -0.04489765 -0.01277639
  0.10506515 -0.04058572  0.01650862  0.07378402  0.04254852  0.06108569
  0.11716671  0.07182718  0.04443775 -0.0451146   0.15268535 -0.02219777
  0.05580912  0.00991383  0.09396805  0.01950088  0.08588489]


In [45]:
print(np.shape(Z))
print(np.shape(K))
print(np.shape(f))

(35, 63)
(63, 63)
(63,)


In [52]:
#Lets now hitch these values onto a new dataframe of skills:

output_df = pd.DataFrame({"Skills":df2.columns,"Importance":E})

In [55]:
output_df = output_df.sort_values(by="Importance")
output_df.reset_index(drop=True, inplace = True)
print(output_df)

                               Skills  Importance
0                     Active Learning   -0.170356
1                             Writing   -0.161264
2                          Persuasion   -0.098492
3                 Service Orientation   -0.058743
4            Quality Control Analysis   -0.045115
5             Complex Problem Solving   -0.044898
6                 Equipment Selection   -0.040586
7                        Coordination   -0.027435
8                    Systems Analysis   -0.022198
9                          Monitoring   -0.021743
10                Operations Analysis   -0.012776
11                           Speaking   -0.006605
12                        Instructing    0.002810
13                    Time Management    0.009914
14                       Installation    0.016509
15   Management of Material Resources    0.019501
16                        Mathematics    0.024968
17              Reading Comprehension    0.028668
18                        Negotiation    0.035196


In [59]:
output_df.to_csv("PIGEBaQ_Ex.csv")